In [8]:
import os, re
import logging

In [35]:
basepath = "../../resources/dictionary_cap/"

In [86]:
# read in the search strings into a list of lists
topics = []
fcount = 0

for filename in os.listdir(basepath):
    fcount += 1
    if filename.endswith(".txt"): 
        with open(os.path.join(basepath, filename)) as fi:
            for line in fi.readlines():
                topics.append(line.strip().split('\n'))
     #   print(os.path.join(basepath, filename))

In [73]:
topic1 = []
with open('../../resources/dictionary_cap/t100.txt') as fi:
    for line in fi.readlines():
        topic1.append(line.strip().split('\n'))
print("Created analyzer with {} words in the dictionary".format(len(topic1)))

Created analyzer with 84 words in the dictionary


In [68]:
class dict_topic_classifier():
    '''This class provides an Analyzer of topics based on dictionaries.'''

    def __init__(self):
        self.nmodel = 0
        
    def topic_classifier(self, topic, occ = 2):
        self.topic = topic
        self.occ = occ
        for document in get_data(): # read in data
            match = [x for x in topic1 if x in document]
            print(match)
            if len(match) > occ:
                print("topic contains the following topic : {}".format(topic))
                